In [36]:
import os
os.environ["http_proxy"] = "http://127.0.0.1:8889"
os.environ["https_proxy"] = "http://127.0.0.1:8889"

In [61]:
from transformers import AutoTokenizer, AutoModelForMaskedLM, DataCollatorForLanguageModeling, Trainer, TrainingArguments
from datasets import load_dataset, DatasetDict, Dataset
import datasets

In [62]:
# ds = load_dataset("pleisto/wikipedia-cn-20230720-filtered")
ds = DatasetDict.load_from_disk("datasets/wikipedia-cn-20230720-filtered")
ds = Dataset.from_dict(ds["train"][:10000])


/home/dean/miniconda3/envs/llm/lib/python3.11/site-packages/datasets/table.py:1421: FutureWarning: promote has been superseded by mode='default'.
  table = cls._concat_blocks(blocks, axis=0)


In [64]:
ds[:3]

{'source': ['wikipedia.zh2307', 'wikipedia.zh2307', 'wikipedia.zh2307'],
 'completion': ['昭通机场（ZPZT）是位于中国云南昭通的民用机场，始建于1935年，1960年3月开通往返航班“昆明－昭通”，原来属军民合用机场。1986年机场停止使用。1991年11月扩建，于1994年2月恢复通航。是西南地区「文明机场」，通航城市昆明。 机场占地1957亩，飞行区等级为4C，有一条跑道，长2720米，宽48米，可供波音737及以下机型起降。机坪面积6600平方米，停机位2个，航站楼面积1900平方米。位于城东6公里处，民航路与金鹰大道交叉处。\n航点\n客服电话\n昭通机场客服电话：0870-2830004',
  '我的英雄学院：英雄新世纪\n《我的英雄学院剧场版：英雄新世纪》（仆のヒーローアカデミア THE MOVIE ヒーローズ:ライジング）是一部于2019年12月20日上映的日本动画电影，由长崎健司执导、黑田洋介编剧，改编自日本漫画家堀越耕平创作的漫画系列《我的英雄学院》，同时也是其系列第二部电影版。\n概要\n本作电影内容同样为作者堀越耕平监修的原创故事，并表示「这部电影版某种意义上可以说是《我的英雄学院》的结局了」。\n登场角色\n制作人员\n主题曲\n 「ハイヤーグラウンド」\n 作词：片冈健太，作曲：黑田隼之介，主唱：sumika\n跨媒体展开\n集英社亦推出该片的小说版和文库版，由小说家誉司アンリ执笔著作，小说版于2019年12月20日上市。\n 仆のヒーローアカデミア THE MOVIE ヒーローズ:ライジング\n 仆のヒーローアカデミア\u3000THE\u3000MOVIE\u3000ヒーローズ : ライジング\u3000ノベライズ\u3000みらい文库版',
  '黄大仙文化公园（Wong Tai Sin Culture Park）是香港一个公园，位于九龙黄大仙摩士公园，门牌编号为香港黄大仙区竹园大成街8号，为一个公园中的公园，入口设于东头村道及大成街。公园原址为摩士二号公园的苗圃，后来由黄大仙区民政事务总署拨款1,370万港元，改建为一个以中国文化为主题的公园，于2008年10月落成启用。\n设施\n* 无极广场：铺砌《易经》、八卦及太极符号\n* 诗墙：刻

In [65]:
tokenizer = AutoTokenizer.from_pretrained("hfl/chinese-macbert-base")

def process_func(example):
    return tokenizer(example["completion"], max_length=384, truncation=True)

In [66]:
tokenized_ds = ds.map(process_func, remove_columns=ds.column_names)
tokenized_ds

Map: 100%|██████████| 10000/10000 [00:08<00:00, 1184.10 examples/s]


Dataset({
    features: ['input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 10000
})

In [67]:
from torch.utils.data import DataLoader

dl = DataLoader(tokenized_ds, batch_size=8, collate_fn=DataCollatorForLanguageModeling(tokenizer, mlm=True, mlm_probability=0.15))

In [68]:
next(enumerate(dl))

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


(0,
 {'input_ids': tensor([[ 101, 3220, 6858,  ...,    0,    0,    0],
         [ 101, 2769, 4638,  ...,    0,    0,    0],
         [ 101, 7942, 1920,  ...,    0,    0,    0],
         ...,
         [ 101, 3452, 4065,  ...,  103,  671,  102],
         [ 101,  517, 2358,  ...,    0,    0,    0],
         [ 101,  818, 1814,  ..., 8021, 8024,  102]]), 'token_type_ids': tensor([[0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         ...,
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         ...,
         [1, 1, 1,  ..., 1, 1, 1],
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 1, 1, 1]]), 'labels': tensor([[-100, -100, -100,  ..., -100, -100, -100],
         [-100, -100, -100,  ..., -100, -100, -100],
         [-100, -100, -100,  ..., -100, -100, -

In [69]:
model = AutoModelForMaskedLM.from_pretrained("hfl/chinese-macbert-base")

Some weights of the model checkpoint at hfl/chinese-macbert-base were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'bert.pooler.dense.bias', 'cls.seq_relationship.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [70]:
training_args = TrainingArguments(
    output_dir="./models/macbert",
    per_device_train_batch_size=32,
    logging_steps=100,
    num_train_epochs=1,
)

In [71]:
trainer = Trainer(
    args=training_args,
    model=model,
    train_dataset=tokenized_ds,
    data_collator=DataCollatorForLanguageModeling(tokenizer, mlm=True, mlm_probability=0.15),
)

In [72]:
model.device

device(type='cuda', index=0)

In [73]:
trainer.train()

Step,Training Loss
100,1.398400
200,1.415600
300,1.391500
400,1.391800
500,1.348600
600,1.367700
700,1.339700
800,1.386500
900,1.326400
1000,1.322400


TrainOutput(global_step=1250, training_loss=1.36167939453125, metrics={'train_runtime': 229.0518, 'train_samples_per_second': 43.658, 'train_steps_per_second': 5.457, 'total_flos': 1972927327269504.0, 'train_loss': 1.36167939453125, 'epoch': 1.0})

In [74]:
from transformers import pipeline

pipe = pipeline("fill-mask", model=model, tokenizer=tokenizer, device=0)

In [86]:
pipe("下面是一个[MASK]面评价：这本书真不错啊。")

/home/dean/miniconda3/envs/llm/lib/python3.11/site-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


[{'score': 0.6843450665473938,
  'token': 741,
  'token_str': '书',
  'sequence': '下 面 是 一 个 书 面 评 价 ： 这 本 书 真 不 错 啊 。'},
 {'score': 0.042484719306230545,
  'token': 6566,
  'token_str': '负',
  'sequence': '下 面 是 一 个 负 面 评 价 ： 这 本 书 真 不 错 啊 。'},
 {'score': 0.025304028764367104,
  'token': 2190,
  'token_str': '对',
  'sequence': '下 面 是 一 个 对 面 评 价 ： 这 本 书 真 不 错 啊 。'},
 {'score': 0.023490050807595253,
  'token': 677,
  'token_str': '上',
  'sequence': '下 面 是 一 个 上 面 评 价 ： 这 本 书 真 不 错 啊 。'},
 {'score': 0.018813038244843483,
  'token': 3633,
  'token_str': '正',
  'sequence': '下 面 是 一 个 正 面 评 价 ： 这 本 书 真 不 错 啊 。'}]